## Database Host Models

In [ ]:
#| default_exp db_host

In [ ]:
#| export

from fastsql import *
from fastcore.utils import *
import uuid
import os
import urllib.parse
from datetime import datetime

In [ ]:
#| export

def timestamp(): return datetime.utcnow().isoformat()
def gen_id(): return uuid.uuid4().hex

In [ ]:
#| export

class GlobalUser:
    """Identity: Who is this person?"""
    id: str; email: str; oauth_id: str
    password_hash: str = None; stripe_cust_id: str = None
    is_sys_admin: bool = False; created_at: str; last_login: str = None

class TenantCatalog:
    """Registry: Where is the database?"""
    id: str; name: str; db_url: str
    is_active: bool = True; plan_tier: str = "free"; created_at: str

class Membership:
    """Router: Which tenants can they access?"""
    id: str; user_id: str; tenant_id: str; profile_id: str
    meta_attributes: str = None; role: str = "member"
    is_active: bool = True; created_at: str

class Subscription:
    """Billing: Are they allowed to use the app?"""
    id: str; tenant_id: str
    stripe_sub_id: str; stripe_cust_id: str
    plan_tier: str; status: str
    current_period_end: str; cancel_at_period_end: bool = False

class HostAuditLog:
    """Security: Who changed the system?"""
    id: str; actor_user_id: str; event_type: str
    target_id: str = None; details: str = None; ip_address: str = None
    created_at: str

class SystemJob:
    """Maintenance: Provisioning & Cleanups"""
    id: str; job_type: str; status: str
    payload: str = None; error_log: str = None
    created_at: str; completed_at: str = None



## Singleton

In [ ]:
#| export

class HostDatabase:
    """
    Singleton wrapper for host database connection and table objects.
    """
    _instance = None
    
    def __new__(cls, db_url: str = None):
        """Singleton pattern: only one instance per application"""
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self, db_url: str = None):
        """Initialize database connection and create table objects"""
        if self._initialized:
            return
        
        if db_url is None:
            raise ValueError("db_url required for first initialization")
        
        # Create database connection
        self.db = Database(db_url)
        
        # Create table objects for host schema
        self.global_users = self.db.create(GlobalUser, name="core_users", pk='id')
        self.tenant_catalogs = self.db.create(TenantCatalog, name="core_tenants", pk='id')
        self.memberships = self.db.create(Membership, name="core_memberships", pk='id')
        self.subscriptions = self.db.create(Subscription, name="core_subscriptions", pk='id')
        self.audit_logs = self.db.create(HostAuditLog, name="sys_audit_logs", pk='id')
        self.system_jobs = self.db.create(SystemJob, name="sys_jobs", pk='id')
        
        self._initialized = True
    
    @classmethod
    def from_env(cls):
        """
        Create HostDatabase from environment variables.
        
        Environment Variables:
            DB_TYPE: Database type (POSTGRESQL or SQLITE, default: POSTGRESQL)
            DB_USER: Database user (default: postgres)
            DB_PASS: Database password (required for PostgreSQL)
            DB_HOST: Database host (default: localhost)
            DB_PORT: Database port (default: 5432)
            DB_NAME: Database name (default: app_host)
        
        Returns:
            HostDatabase: Singleton instance
        """
        DB_TYPE = os.getenv("DB_TYPE", "POSTGRESQL")
        DB_USER = os.getenv("DB_USER", "postgres")
        DB_PASS = os.getenv("DB_PASS", "")
        DB_HOST = os.getenv("DB_HOST", "localhost")
        DB_PORT = os.getenv("DB_PORT", "5432")
        DB_NAME = os.getenv("DB_NAME", "app_host")
        
        if DB_TYPE == "POSTGRESQL":
            if not DB_PASS:
                raise ValueError("DB_PASS is required for PostgreSQL")
            encoded_pass = urllib.parse.quote_plus(DB_PASS)
            db_url = f"postgresql://{DB_USER}:{encoded_pass}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
        else:
            db_url = f"sqlite:///{DB_NAME}.db"
        
        return cls(db_url)
    
    def commit(self):
        """Commit current transaction"""
        self.db.conn.commit()
    
    def rollback(self):
        """Rollback current transaction"""
        self.db.conn.rollback()
    
    @classmethod
    def reset_instance(cls):
        """Reset singleton (for testing only)"""
        cls._instance = None